In [1]:
# 필요한 모듈과 라이브러리를 로딩
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time, os, math, random, sys, datetime
from datetime import date, timedelta, datetime
import pyautogui
from tqdm import tqdm
import requests, re

# chromedriver 자동 설치
import chromedriver_autoinstaller

from selenium.webdriver.common.action_chains import ActionChains


In [37]:
# 작업 시간과 고유 dir 등 생성
n = time.localtime()
s = '%04d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday)
s_time = time.time( )

page_num = 5993

a = datetime.now()-timedelta(days=42)
a = a.strftime('%Y-%m-%d')
b = datetime.now()-timedelta(days=43)
b = b.strftime('%Y-%m-%d')
print(b, a)

2021-04-21 2021-04-22


In [3]:
print("=" *80)
print("         FM 정치/시사 게시판 크롤러입니다.")
print("=" *80)

f_dir = os.path.expanduser('~') + f'\\Desktop\\daily\\{b}-{a}\\' # 운영체제 상관없이 바탕화면 'crawler-bot' 폴더에 작업

print("=" *80)
print("데이터 크롤링을 시작합니다.")
print("=" *80)

         FM 정치/시사 게시판 크롤러입니다.
데이터 크롤링을 시작합니다.


In [4]:
# 저장될 파일 경로와 이름을 지정
# 파일을 저장할 폴더 위치를 만들고, 중복일 경우 해당 폴더를 삭제하고 새로 생성합니다.
try : 
    os.makedirs(f_dir)
except : pass
else : 
    os.chdir(f_dir)

fc_name=f_dir+'\\'+f'fm.csv'

In [5]:
# 크롤링 데이터가 들어갈 리스트 생성
domain2 = []    # 크롤링 도메인
title2 = []     # 게시물 제목
user2 = []      # 게시물 작성자
url2 = []       # 게시물 url
time2 = []      # 게시물 작성 시간
view_cnt2 = []  # 게시물 뷰 카운트
text2 = []      # 게시물 내용
com_cnt2 = []   # 댓글 수
com_user2 = []  # 작성자 본인을 제외한 댓글 유저
com_text2 = []  # 댓글 user_id & text

In [17]:
# chromedriver-autoinstaller 이용
chromedriver_autoinstaller.install(cwd=True)

# HEADLESS MODE
options = webdriver.ChromeOptions()
# options.add_argument('headless')
options.add_argument('window-size=1920x1080') 
# options.add_argument("--disable-gpu")
options.add_experimental_option('excludeSwitches', ['enable-logging'])
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")
args = ["hide_console", ]

# 알림창 끄기
options.add_experimental_option("prefs", {"profile.default_content_setting_values.notifications": 1})

# 웹사이트 접속 후 해당 메뉴로 이동
driver = webdriver.Chrome(options=options,service_args=args)
driver.maximize_window()

In [7]:
domain1 = "fm"
print("==========",domain1,"크롤링 진행 중 ==========")

========== fm 크롤링 진행 중 ==========


In [39]:

while_code = 0

while True :
    if while_code == 1 :
        print("1차 데이터 수집 완료")
        print("2차 데이터 수집으로 넘어갑니다.")
        break
    
    driver.get(f'https://www.fmkorea.com/index.php?mid=news&page={page_num}')
    WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, "/html/body/div/div/div/div/div/div/div/div/table/thead")))        
    time.sleep(1)
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    post_box = soup.find('tbody').find_all('tr', class_=False)
    
    for post in post_box :
        
        post_url = "https://www.fmkorea.com/" + post.find("td", class_=re.compile('title')).find('a')['href']
        print("url :", post_url)
#         req2 = requests.get(post_url)
        driver.get(post_url)
        WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#bd_capture")))        
        time.sleep(1)
        html2 = driver.page_source
#         html2 = req2.text
        soup2 = BeautifulSoup(html2, 'lxml')
#         print(soup2.select('#bd_4180795_3604166605 > div.rd.rd_nav_style2.clear'))
#         print("aa:", soup2.find('span', class_='date m_no'))
#         print("bb:", soup2.find('span', class_='date m_no').text)
        post_date = soup2.find('span', class_='date m_no').text[:10].replace(".","-")
        
        if post_date > a :
#             page_num += 1
            print(post_date, " > ", a)
            print("페이지를 넘깁니다.\n")
            break
            
        elif post_date >= b :
            if post_url not in url2 :

                domain1 = domain1
                title1 = soup2.find('span', class_='np_18px_span').text
                user1 = soup2.find('a', class_=re.compile('nick member')).text
                url1 = post_url
                view1 = soup2.find_all(class_='side fr')[0].find('b').text
                try :
                    com_cnt1 = soup2.find(class_='ui_font bubble').find('b').text
                except :
                    com_cnt1 = 0

                text1 = soup2.find('article').text
                com_user1 = []
                com_text1 = []

                try :

                    com_box = soup2.find(class_='fdb_lst_ul').find_all('li')

                    for com in com_box :

                        com_user = com.find('a', class_=re.compile('member_')).text

                        try :
                            delete = com.find('a', class_='findParent').text
                            comment = com.find('div', class_=re.compile('comment_')).text
                            comment.replace(delete, "")
                        except :
                            comment = com.find('div', class_=re.compile('comment_')).text

                        com_user1.append(com_user)
                        com_text1.append(f"작성자 : {com_user} :: {comment}")
                except :
                    com_user1.append("")
                    com_text1.append("")       



    #             print("게시일 :", post_date)
    #             print("제목 :", title1)
    #             print("작성자 :", user1)
    #             print("url :", url1)
    #             print("조회수 :", view1)
    #             print("게시물 내용 :", text1)
    #             print("댓글수 :", com_cnt1)
    #             print("댓글 작성자 :", com_user1)
    #             print("댓글 내용 :", com_text1)
    #             print("\n")

                domain2.append(domain1)
                title2.append(title1)
                time2.append(post_date)
                user2.append(user1)
                url2.append(url1)
                view_cnt2.append(view1)
                com_cnt2.append(com_cnt1)
                text2.append(text1)
                com_user2.append(com_user1)
                com_text2.append(com_text1)

    #             time.sleep(2)

            
        else :
            while_code = 1
            break

    page_num += 1
    driver.delete_all_cookies()


url : https://www.fmkorea.com//3541323189
url : https://www.fmkorea.com//3541317978
url : https://www.fmkorea.com//3541313045
url : https://www.fmkorea.com//3541317884
url : https://www.fmkorea.com//3541316640
url : https://www.fmkorea.com//3541315811
url : https://www.fmkorea.com//3541315651
url : https://www.fmkorea.com//3541313476
url : https://www.fmkorea.com//3541311164
url : https://www.fmkorea.com//3541312025
url : https://www.fmkorea.com//3541310379
url : https://www.fmkorea.com//3541309693
url : https://www.fmkorea.com//3541309277
url : https://www.fmkorea.com//3541307987
url : https://www.fmkorea.com//3541304046
url : https://www.fmkorea.com//3541302942
url : https://www.fmkorea.com//3541299516
url : https://www.fmkorea.com//3541299480
url : https://www.fmkorea.com//3541299079
url : https://www.fmkorea.com//3541296733
url : https://www.fmkorea.com//3541316640
url : https://www.fmkorea.com//3541315811
url : https://www.fmkorea.com//3541315651
url : https://www.fmkorea.com//354

url : https://www.fmkorea.com//3541207384
url : https://www.fmkorea.com//3541206700
url : https://www.fmkorea.com//3541204835
url : https://www.fmkorea.com//3541204828
url : https://www.fmkorea.com//3541216787
url : https://www.fmkorea.com//3541217288
url : https://www.fmkorea.com//3541216267
url : https://www.fmkorea.com//3541215333
url : https://www.fmkorea.com//3541212167
url : https://www.fmkorea.com//3541210992
url : https://www.fmkorea.com//3541209491
url : https://www.fmkorea.com//3541207384
url : https://www.fmkorea.com//3541206700
url : https://www.fmkorea.com//3541204835
url : https://www.fmkorea.com//3541204828
url : https://www.fmkorea.com//3541204137
url : https://www.fmkorea.com//3541179205
url : https://www.fmkorea.com//3541200617
url : https://www.fmkorea.com//3541201823
url : https://www.fmkorea.com//3541200687
url : https://www.fmkorea.com//3541197347
url : https://www.fmkorea.com//3541194500
url : https://www.fmkorea.com//3541196863
url : https://www.fmkorea.com//354

url : https://www.fmkorea.com//3541090305
url : https://www.fmkorea.com//3541097726
url : https://www.fmkorea.com//3541097667
url : https://www.fmkorea.com//3541095373
url : https://www.fmkorea.com//3541093995
url : https://www.fmkorea.com//3541093316
url : https://www.fmkorea.com//3541092645
url : https://www.fmkorea.com//3541092504
url : https://www.fmkorea.com//3541097667
url : https://www.fmkorea.com//3541095373
url : https://www.fmkorea.com//3541093995
url : https://www.fmkorea.com//3541093316
url : https://www.fmkorea.com//3541092645
url : https://www.fmkorea.com//3541092504
url : https://www.fmkorea.com//3541092140
url : https://www.fmkorea.com//3541091637
url : https://www.fmkorea.com//3541089947
url : https://www.fmkorea.com//3541088819
url : https://www.fmkorea.com//3541088949
url : https://www.fmkorea.com//3541087749
url : https://www.fmkorea.com//3541087603
url : https://www.fmkorea.com//3541086132
url : https://www.fmkorea.com//3541086152
url : https://www.fmkorea.com//354

url : https://www.fmkorea.com//3540769116
url : https://www.fmkorea.com//3540766906
url : https://www.fmkorea.com//3540765835
url : https://www.fmkorea.com//3540762402
url : https://www.fmkorea.com//3540758729
url : https://www.fmkorea.com//3540758063
url : https://www.fmkorea.com//3540743092
url : https://www.fmkorea.com//3540748811
url : https://www.fmkorea.com//3540743003
url : https://www.fmkorea.com//3540740276
url : https://www.fmkorea.com//3540736074
url : https://www.fmkorea.com//3540735031
url : https://www.fmkorea.com//3540758729
url : https://www.fmkorea.com//3540758063
url : https://www.fmkorea.com//3540743092
url : https://www.fmkorea.com//3540748811
url : https://www.fmkorea.com//3540743003
url : https://www.fmkorea.com//3540740276
url : https://www.fmkorea.com//3540736074
url : https://www.fmkorea.com//3540735031
url : https://www.fmkorea.com//3540730294
url : https://www.fmkorea.com//3540729786
url : https://www.fmkorea.com//3540711351
url : https://www.fmkorea.com//354

url : https://www.fmkorea.com//3540564816
url : https://www.fmkorea.com//3540564085
url : https://www.fmkorea.com//3540563501
url : https://www.fmkorea.com//3540561071
url : https://www.fmkorea.com//3540557327
url : https://www.fmkorea.com//3540557239
url : https://www.fmkorea.com//3540555098
url : https://www.fmkorea.com//3540554298
url : https://www.fmkorea.com//3540553771
url : https://www.fmkorea.com//3540552566
url : https://www.fmkorea.com//3540549011
url : https://www.fmkorea.com//3540549352
url : https://www.fmkorea.com//3540547031
url : https://www.fmkorea.com//3540546813
url : https://www.fmkorea.com//3540545898
url : https://www.fmkorea.com//3540545868
url : https://www.fmkorea.com//3540553771
url : https://www.fmkorea.com//3540552566
url : https://www.fmkorea.com//3540549011
url : https://www.fmkorea.com//3540549352
url : https://www.fmkorea.com//3540547031
url : https://www.fmkorea.com//3540546813
url : https://www.fmkorea.com//3540545898
url : https://www.fmkorea.com//354

url : https://www.fmkorea.com//3540434295
url : https://www.fmkorea.com//3540433202
url : https://www.fmkorea.com//3540419990
url : https://www.fmkorea.com//3540423353
url : https://www.fmkorea.com//3540425833
url : https://www.fmkorea.com//3540428422
url : https://www.fmkorea.com//3540428131
url : https://www.fmkorea.com//3540426679
url : https://www.fmkorea.com//3540424364
url : https://www.fmkorea.com//3540423932
url : https://www.fmkorea.com//3540423874
url : https://www.fmkorea.com//3540423605
url : https://www.fmkorea.com//3540422119
url : https://www.fmkorea.com//3540421880
url : https://www.fmkorea.com//3540409363
url : https://www.fmkorea.com//3540409596
url : https://www.fmkorea.com//3540401525
url : https://www.fmkorea.com//3540407256
url : https://www.fmkorea.com//3540406603
url : https://www.fmkorea.com//3540406063
url : https://www.fmkorea.com//3540407256
url : https://www.fmkorea.com//3540406603
url : https://www.fmkorea.com//3540406063
url : https://www.fmkorea.com//354

In [ ]:
print("Let's Test")

In [40]:
df = pd.DataFrame()

df['domain'] = pd.Series(domain2)
# df['keyword'] = pd.Series(keyword2)
df['title'] = pd.Series(title2)
df['user_name'] = pd.Series(user2)
df['url'] = pd.Series(url2)
df['date'] = pd.Series(time2)
df['view_cnt'] = pd.Series(view_cnt2)
# df['like_cnt'] = pd.Series(like_cnt2)
df['user_text'] = pd.Series(text2)
df['com_cnt'] = pd.Series(com_cnt2)
df['com_user'] = pd.Series(com_user2)
df['com_text'] = pd.Series(com_text2)

df

,domain,title,user_name,url,date,view_cnt,user_text,com_cnt,com_user,com_text
0,fm,페미가 이익을 챙길수록 장애인의 권리를 잃어가는거 같음,rewre,https://www.fmkorea.com//3538619111,2021-04-20,88,1. 주차장2. 기업 가산점3. 노동,1,[파괴왕김파괴],[작성자 : 파괴왕김파괴 :: 창업관련 가산점제에서 장애인가산점 없애고 여성가산점 ...
1,fm,오늘자 안철수 근황.JPG,사우론눈까리,https://www.fmkorea.com//3538617786,2021-04-20,14100,카이스트 신형원자로 투자하러 갔음.,34,"[날개3, Plue, Centrism, 시럽맞좀볼래, 주접극장, 국민의적, 원자력햄...","[작성자 : 날개3 :: 국가의 미래생각하는 사람은 안철수 밖에 없다., 작성자 :..."
2,fm,소드 쭉빵 애들 오세훈 유튜브 영상마다 좌표찍네 ㅋㅋ,ㅈ문가,https://www.fmkorea.com//3538618587,2021-04-20,226,이거 괜찮은거냐?,2,"[태사다르, 5세훈]",[작성자 : 태사다르 :: 조회수만 올려주고 좋지 ㅋㅋ어차피 오세훈 유튜브 보는 사...
3,fm,제발 97년생부터 여자 징병 제발,축구용사페레즈,https://www.fmkorea.com//3538617390,2021-04-20,123,누나 메갈년 제발 씨발,7,"[봄이여오라!, 김휘집, 거침없이하이킥, 오렝지렌지, 범선, 윤서굴리트결, 推薦]","[작성자 : 봄이여오라! :: ㅋㅋㅋㅋ, 작성자 : 김휘집 :: 아 ㅋㅋ, 작성자 ..."
4,fm,이 대깨문 새끼야 이것부터 설명해봐 ㅋㅋㅋㅋㅋㅋㅋㅋㅋ,길거리투팍,https://www.fmkorea.com//3538609977,2021-04-20,875,ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ 이 병신은 이거 설명할수 있어?광주/전라는 지들 땅...,7,"[파괴왕김파괴, 길거리투팍, 착쁜펨코, 착쁜펨코, 제라드성님, 일라이매닝, 옥수수한...",[작성자 : 파괴왕김파괴 :: 애국보수 대구학식충으로서 저런 댓글 못참는다 링크 ㄱ...
...,...,...,...,...,...,...,...,...,...,...
14340,fm,창원대 철학과 윤김지영 교수님의 여성징병제에 대한 의견,할게없,https://www.fmkorea.com//3540373657,2021-04-21,216,\n.,3,"[얼죽아아, 무식은mylif, 코스톨라니]","[작성자 : 얼죽아아 :: 남자들은 부조리 있다고 안끌려감? ㅋㅋㅋ, 작성자 : 무..."
14341,fm,미국이랑 백신 스와프 될 경우 아스트라 일수도 있단다 ㅅㅂ,rlals01,https://www.fmkorea.com//3540374117,2021-04-21,235,\nhttps://n.news.naver.com/article/020/0003352...,1,[야자나무],[작성자 : 야자나무 :: 시발 차라리 거부]
14342,fm,국힘 당원 가입 인증,앙재앙띠,https://www.fmkorea.com//3540371926,2021-04-21,3295,\n20대 목소리 좀 들어줘라 슈발람들아,5,"[5세훈, 냉철하게살자, 아스클레피오스, Cmaj7, 익꾸]","[작성자 : 5세훈 :: ㅊㅊ, 작성자 : 냉철하게살자 :: ㅊㅊ, 작성자 : 아스..."
14343,fm,가카였으면 백신접종률 3손가락 안에 들었다 ㄹㅇ루다가,안정성의섬,https://www.fmkorea.com//3540370703,2021-04-21,233,"바이든한테 전화해서 바로 받아왔음아니다 오히려 바이든이 전화걸어서 ""브라더"" 한번 ...",2,"[재앙의시대, 안정성의섬]",[작성자 : 재앙의시대 :: 명박이라면 그걸로 뒷돈 챙겼지 절대 안사올 사람은 아니...


In [19]:
page_num = 6472

In [41]:
for i in range(len(df)) :
    if len(df['user_text'][i]) < 5 :
        df['user_text'][i] = df['title'][i]

In [43]:
df['date'].unique()

array(['2021-04-20', '2021-04-19', '2021-04-18', '2021-04-17',
       '2021-04-16', '2021-04-21'], dtype=object)

In [42]:
# csv 형태로 저장하기
df.to_csv(fc_name,encoding="utf-8-sig",index=False)

e_time = time.time( )
t_time = e_time - s_time

print("\n")
print("=" *80)
print("1.총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("2.파일 저장 완료: csv 파일명 : %s " %fc_name)

driver.quit()
sys.exit("프로그램을 종료합니다.")



1.총 소요시간은 7753.5 초 입니다 
2.파일 저장 완료: csv 파일명 : C:\Users\Qscar KIM\Desktop\daily\2021-04-16-2021-04-22\\fm.csv 


SystemExit: 프로그램을 종료합니다.

c:\python38\lib\site-packages\IPython\core\interactiveshell.py:3426: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## 